# Chapter 9 연습문제와 종합문제 해답

# Chapter 9 연습문제 해답

In [ ]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import sklearn

%precision 3

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

X, y = make_blobs(random_state=52)
plt.scatter(X[:,0], X[:,1], color='black')

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

iris = load_iris()

sc = StandardScaler()
sc.fit(iris.data)
X_std = sc.transform(iris.data)

# 주성분 분석 실행
pca = PCA(n_components=2)
pca.fit(X_std)
X_pca = pca.transform(X_std)

print('주성분 분석 전 데이터 차원:{}'.format(iris.data.shape))
print('주성분 분석 후 데이터 차원:{}'.format(X_pca.shape))

In [ ]:
merge_data = pd.concat([pd.DataFrame(X_pca[:,0]), pd.DataFrame(X_pca[:,1]), pd.DataFrame(iris.target)], axis=1)
merge_data.columns = ['pc1','pc2', 'target']

# 군집결과 시각화
ax = None
colors = ['blue', 'red', 'green']
for i, data in merge_data.groupby('target'):
    ax = data.plot.scatter(
        x='pc1', y='pc2',
        color=colors[i], label=f'target-{i}', ax=ax
    )

In [ ]:
# 목표변수와 4개의 설명변수 간의 관계 시각화
fig, axes = plt.subplots(2,2,figsize=(20,7))

iris_0 = iris.data[iris.target==0]
iris_1 = iris.data[iris.target==1]
iris_2 = iris.data[iris.target==2]

ax = axes.ravel()
for i in range(4):
    _,bins = np.histogram(iris.data[:,i],bins=50)
    ax[i].hist(iris_0[:,i],bins=bins,alpha=.5)
    ax[i].hist(iris_1[:,i],bins=bins,alpha=.5)
    ax[i].hist(iris_2[:,i],bins=bins,alpha=.5)
    ax[i].set_title(iris.feature_names[i])
    ax[i].set_yticks(())
ax[0].set_ylabel('Count')
ax[0].legend(['setosa','versicolor','virginica'], loc='best')
fig.tight_layout()

# Chapter 9 종합문제 해답

In [ ]:
# 구매 이력 데이터 읽어 들이기(cd ./chap9 명령어로 현재 디렉터리를 데이터가 있는 디렉터리로 이동하고 실행합니다)
trans = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')
trans.head()

In [ ]:
trans['cancel_flg'] = trans.InvoiceNo.map(lambda x:str(x)[0])
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]

In [ ]:
import itertools

# 행 수가 1,000보다 큰 StockCode 추출
indexer = trans.StockCode.value_counts() > 1000
Items = trans.StockCode.value_counts()[indexer.index[indexer]].index

# 집계 대상 데이터에 포함된 InvoiceNo 수 계산(지지도 계산 시 분모)
trans_all = set(trans.InvoiceNo)

# 대상 Items에 포함된 임의의 2개 StockCode에 대한 지지도 계산
results={}
for element in itertools.combinations(Items, 2):
    trans_0 = set(trans[trans['StockCode']==element[0]].InvoiceNo)
    trans_1 = set(trans[trans['StockCode']==element[1]].InvoiceNo)
    trans_both = trans_0&trans_1
    
    support = len(trans_both) / len(trans_all)
    results[element] = support

maxKey =  max([(v,k) for k,v in results.items()])[1]
print('지지도가 최대인 StockCode 조합：{}'.format(maxKey))
print('지지도 최댓값：{:.4f}'.format(results[maxKey]))